In [183]:
from vpic import TypedClient
from vpic import Client
from thefuzz import fuzz
from thefuzz import process

In [184]:
clt = Client()

In [185]:
import pandas as pd

In [186]:
apple = pd.read_csv("apple.csv")

In [187]:
apple.head(25)

,brand,full_name,short,start,end
0,abarth,2017 - 2023 Abarth 595,Abarth 595,2017.0,2023.0
1,abarth,2017 - 2023 Abarth 695,Abarth 695,2017.0,2023.0
2,abarth,2023 Abarth 500e,Abarth 500e,2023.0,-1.0
3,acura,2017 - 2023 NSX,NSX,2017.0,2023.0
4,acura,2018 - 2023 MDX,MDX,2018.0,2023.0
5,acura,2018 - 2023 TLX,TLX,2018.0,2023.0
6,acura,2019 - 2022 ILX,ILX,2019.0,2022.0
7,acura,2019 - 2023 RDX,RDX,2019.0,2023.0
8,acura,2023 Integra,Integra,2023.0,-1.0
9,acura,2024 ZDX,ZDX,2024.0,-1.0


In [188]:
brands = set(apple["brand"])

In [189]:
len(brands)

82

In [190]:
def rename(brand):
    '''
    note certain names don't produce queries but they are still present
    '''
    if brand == "vw":
        return "volkswagen"
    
    if brand == "polaris slingshot":
        return "polaris"
    
    return brand

In [191]:
prev_brand = "definitely not a car brand"
results = []
results_dict = {}
makeids = []
modelids = []
fuzzy_scores = []
model_names = []
matched_names = []
m_year_first = [] # first time a model in this category supported CarPlay

for i in range (len(apple)):

    brand = apple["brand"][i]
    model = apple["short"][i].lower() # avoid case sensitivity
    start = apple["start"][i]

    if "-" in brand:
        brand = " ".join(brand.split("-"))

    brand = rename(brand)

    if brand != prev_brand:
        results = clt.get_models_for_make(brand)
        results_dict = {}
        for result in results:
            results_dict[result["Model"].lower()] = result["ModelId"]
        prev_brand = brand
        model_names = results_dict.keys()
    
    if len(results) > 0:
        makeids.append(results[0]["MakeId"])
        choice, score = process.extractOne(model, model_names)
        fuzzy_scores.append(score)
        modelids.append(results_dict[choice])
        matched_names.append(choice)
        # if model in results_dict.keys():
        #     modelids.append(results_dict[model])
        # else:
        #     model = model.split(" ")[0]
        #     if model in results_dict.keys():
        #         modelids.append(results_dict[model])
        #     else:
        #         modelids.append(-1)

    else:
        fuzzy_scores.append(-1)
        makeids.append(-1)
        modelids.append(-1)
        matched_names.append("not matched")

In [192]:
len([id for id in modelids if id != -1])

726

In [193]:
apple["modelids"] = modelids
apple["makeids"] = makeids
apple["fuzzy_score"] = fuzzy_scores
apple["matched_name"] = matched_names

In [194]:
apple.to_csv('apple_vin.csv', index=False)

* a bit of work to be done by hand later - a lot are not matched 
* some of which are to be expected; foreign producers
* others may be due to funky naming things

---